<a href="https://colab.research.google.com/github/LeoFernanndes/data_engineering/blob/main/OlistMysqlDataWarehouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mysql-connector-python
!pip install pymysql
!pip install tweepy
!pip install SQLAlchemy

     |████████████████████████████████| 18.0MB 203kB/s 
     |████████████████████████████████| 51kB 2.7MB/s 


In [2]:
import pandas as pd
import json
import mysql.connector
import os
import tweepy as tw
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import time

In [3]:
def mysql_rds_database_authentication(database_credentials, database=None):
  
  if database==None:

    mydb = mysql.connector.connect(
      host = database_credentials['host'],
      user = database_credentials['user'],
      port = database_credentials['port'],
      password = database_credentials['password']
    )
    
  else:
    mydb = mysql.connector.connect(
      host = database_credentials['host'],
      user = database_credentials['user'],
      port = database_credentials['port'],
      password = database_credentials['password'],
      database = database
    )

  return mydb

In [4]:
# olist_customers_dataset.csv
customers_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_customers_dataset.csv'
customers_dataset = pd.read_csv(customers_dataset_url)

# olist_geolocation_dataset.csv
geolocation_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_geolocation_dataset.csv'
geolocation_dataset = pd.read_csv(geolocation_dataset_url)

# olist_order_items_dataset.csv
order_items_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_order_items_dataset.csv'
order_items_dataset = pd.read_csv(order_items_dataset_url)

# olist_order_payments_dataset.csv
order_payments_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_order_payments_dataset.csv'
order_payments_dataset = pd.read_csv(order_payments_dataset_url)

# olist_order_reviews_dataset.csv
order_reviews_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_order_reviews_dataset.csv'
order_reviews_dataset = pd.read_csv(order_reviews_dataset_url)
order_reviews_dataset = order_reviews_dataset.drop(columns=['Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'])

# olist_orders_dataset.csv
orders_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_orders_dataset.csv'
orders_dataset = pd.read_csv(orders_dataset_url)

# olist_products_dataset.csv
products_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_products_dataset.csv'
products_dataset = pd.read_csv(products_dataset_url)

# olist_sellers_dataset.csv
sellers_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_sellers_dataset.csv'
sellers_dataset = pd.read_csv(sellers_dataset_url)

# product_category_name_translation.csv
product_category_name_translation_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/product_category_name_translation.csv'
product_category_name_translation = pd.read_csv(product_category_name_translation_url)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
olist_database_credentials = open("mysql_olist_credentials.json")
olist_database_credentials = json.load(olist_database_credentials)

In [6]:
connection = mysql_rds_database_authentication(olist_database_credentials)

cursor = connection.cursor()
cursor.execute("""
  CREATE DATABASE IF NOT EXISTS olist;
  USE olist;
""")
connection.close()

connection = mysql_rds_database_authentication(olist_database_credentials, 'olist')
cursor = connection.cursor()
cursor.execute("""
  show tables;
""")
tables = cursor.fetchall()
connection.close()

for table in tables:
  print(table[0])

customers
geolocation
geolocation_dataset
order_items
order_payments
order_reviews
orders
product_category_name_translation
products
sellers


In [7]:
# criação das tabelas do banco

connection = mysql_rds_database_authentication(olist_database_credentials, 'olist')
cursor = connection.cursor()

cursor.execute("""
  CREATE TABLE IF NOT EXISTS customers (
    customer_id varchar(100) primary key,
    customer_unique_id varchar(100),
    customer_zip_code_prefix varchar(50),
    customer_city varchar(100),
    customer_state varchar(10)
  );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS geolocation (
    id_num int not null auto_increment primary key,
    geolocation_zip_code_prefix varchar(50),
    geolocation_lat	float,
    geolocation_lng float,
    geolocation_city varchar(100),
    geolocation_state varchar(10)
  );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS order_items (
    id_num int auto_increment primary key,
    order_id varchar(100),
    order_item_id	int,
    product_id varchar(100),
    seller_id varchar(100),
    shipping_limit_date datetime,
    price float,
    freight_value float
  );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS order_payments (
    id_num int auto_increment primary key,
    order_id varchar(100),
    payment_sequential	int,
    payment_type varchar(100),
    payment_installments varchar(100),
    payment_value datetime
  );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS order_reviews (
    id_num int auto_increment primary key,
    review_id varchar(100),
    order_id	varchar(100),
    review_score float,
    review_comment_title varchar(100),
    review_comment_message varchar(255),
    review_creation_date datetime,
    review_answer_timestamp datetime
  );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS order_reviews (
    review_id varchar(100) primary key,
    order_id	varchar(100),
    review_score float,
    review_comment_title varchar(100),
    review_comment_message varchar(255),
    review_creation_date datetime,
    review_answer_timestamp datetime
  );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS orders (
    order_id varchar(100) primary key,
    customer_id	varchar(100),
    order_status varchar(100),
    order_purchase_timestamp datetime,
    order_approved_at datetime,
    order_delivered_carrier_date datetime,
    order_delivered_customer_date datetime,
    order_estimated_delivery_date datetime
  );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS products (
    product_id varchar(100) primary key,
    product_category_name	varchar(100),
    product_name_lenght int,
    product_description_lenght int,
    product_photos_qty float,
    product_weight_g float,
    product_length_cm float,
    product_height_cm float,
    product_width_cm float
  );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS sellers (
    seller_id varchar(100) primary key,
    seller_zip_code_prefix varchar(50),
    seller_city varchar(100),
    seller_state varchar(10)
  );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS product_category_name_translation (
    product_category_name varchar(100) primary key,
    product_category_name_english varchar(100)
  );
""")

In [8]:
connection = mysql_rds_database_authentication(olist_database_credentials, 'olist')
cursor = connection.cursor()
cursor.execute("""
  show tables;
""")
tables = cursor.fetchall()
connection.close()

for table in tables:
  print(table[0])

customers
geolocation
geolocation_dataset
order_items
order_payments
order_reviews
orders
product_category_name_translation
products
sellers


In [9]:
def drop_table(table):
  connection = mysql_rds_database_authentication(olist_database_credentials, 'olist')
  cursor = connection.cursor()
  cursor.execute(f"""
    drop table {table};
  """)

  connection.commit()
  connection.close()

  return f'table {table} successfully removed'


def clear_table(table):
  connection = mysql_rds_database_authentication(olist_database_credentials, 'olist')
  cursor = connection.cursor()
  cursor.execute(f"""
    delete from {table};
  """)
  
  connection.commit()
  connection.close()

  return f'table {table} successfully cleaned'
  

def query_database(query):
  connection = mysql_rds_database_authentication(olist_database_credentials, 'olist')
  dataframe = pd.read_sql(query, con=connection)
  connection.close()

  return dataframe


def mysql_bulk_insert_dataframe(new_data, table, primary_key, mysql_credentials, verbose=False):
  
  try:
    user = mysql_credentials['user']
    password = mysql_credentials['password']
    host = mysql_credentials['host']
    port = mysql_credentials['port']
    database = 'olist'

    engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4")
    database_persisted_primarykeys = pd.read_sql(f'select {primary_key} from {table}', con=engine)
    new_filtered_data = new_data[new_data[primary_key].isin(database_persisted_primarykeys[primary_key]) == False]

    batch_size = 100000
    n_batches = int(np.ceil(new_filtered_data.shape[0] / batch_size))

    for i in range(n_batches):
      if i == n_batches - 1:
        
        first = batch_size * i
        last = new_filtered_data.shape[0]
        
      
      else:

        first = batch_size * i
        last = batch_size * (i + 1)
      
      new_filtered_data.iloc[first: last].to_sql(table, con=engine, if_exists='append', index=False)

      if verbose == True:
        print(f'batch {i+1} successfully inserted on table: {table}')

      time.sleep(2)


    if verbose == True:
      return f'Successfully inserted {new_filtered_data.shape[0]} rows on {table}'

  except Exception as e:
    if verbose == True:
      return f'An error has occurred: {e} => {table}'

In [10]:
# customers
clear_table('customers')
mysql_bulk_insert_dataframe(customers_dataset, 'customers', 'customer_id', olist_database_credentials, verbose=True)

batch 1 successfully inserted on table: customers


'Successfully inserted 99441 rows on customers'

In [11]:
# geolocation
clear_table('geolocation')
mysql_bulk_insert_dataframe(geolocation_dataset, 'geolocation', 'geolocation_zip_code_prefix', olist_database_credentials, verbose=True)

batch 1 successfully inserted on table: geolocation
batch 2 successfully inserted on table: geolocation
batch 3 successfully inserted on table: geolocation
batch 4 successfully inserted on table: geolocation
batch 5 successfully inserted on table: geolocation
batch 6 successfully inserted on table: geolocation
batch 7 successfully inserted on table: geolocation
batch 8 successfully inserted on table: geolocation
batch 9 successfully inserted on table: geolocation
batch 10 successfully inserted on table: geolocation
batch 11 successfully inserted on table: geolocation


'Successfully inserted 1000163 rows on geolocation'

In [12]:
# order_items
clear_table('order_items')
mysql_bulk_insert_dataframe(order_items_dataset, 'order_items', 'order_id', olist_database_credentials, verbose=True)

batch 1 successfully inserted on table: order_items
batch 2 successfully inserted on table: order_items


'Successfully inserted 112650 rows on order_items'

In [13]:
# order_payments
clear_table('order_payments')
mysql_bulk_insert_dataframe(order_payments_dataset, 'order_payments', 'order_id', olist_database_credentials, verbose=True)

batch 1 successfully inserted on table: order_payments
batch 2 successfully inserted on table: order_payments


'Successfully inserted 103886 rows on order_payments'

In [14]:
# order_reviews
clear_table('order_reviews')
mysql_bulk_insert_dataframe(order_reviews_dataset, 'order_reviews', 'review_id', olist_database_credentials, verbose=True)

batch 1 successfully inserted on table: order_reviews


'Successfully inserted 100000 rows on order_reviews'

In [15]:
# orders
clear_table('orders')
mysql_bulk_insert_dataframe(orders_dataset, 'orders', 'order_id', olist_database_credentials, verbose=True)

batch 1 successfully inserted on table: orders


'Successfully inserted 99441 rows on orders'

In [16]:
# products
clear_table('products')
mysql_bulk_insert_dataframe(products_dataset, 'products', 'product_id', olist_database_credentials, verbose=True)

batch 1 successfully inserted on table: products


'Successfully inserted 32951 rows on products'

In [17]:
# sellers
clear_table('sellers')
mysql_bulk_insert_dataframe(sellers_dataset, 'sellers', 'seller_id', olist_database_credentials, verbose=True)

batch 1 successfully inserted on table: sellers


'Successfully inserted 3095 rows on sellers'

In [18]:
# product_category_name_translation
clear_table('product_category_name_translation')
mysql_bulk_insert_dataframe(product_category_name_translation, 'product_category_name_translation', 'product_category_name', olist_database_credentials, verbose=True)

batch 1 successfully inserted on table: product_category_name_translation


'Successfully inserted 71 rows on product_category_name_translation'

In [19]:
query_database('select * from customers')

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS
